# Visual-Area Autolabeler: Training

## About

[Noah C. Benson](mailto:nben@uw.edu)$^{1,2,3}$, [Shaoling Chen](mailto:sc6995@nyu.edu)$^{4}$, and [Jonathan Winawer](mailto:jonathan.winawer@nyu.edu)$^{1,2}$

$^1$Department of Psychology  
$^2$Center for Neural Sciences  
$^4$Courant Institute for Mathematics  
New York University  
New York, NY 10012

$^3$**Current Affiliation:**  
eScience Institute  
University of Washington  
Seattle, WA 98195

## Configuration

Here we define any configuration item that needs to be set locally for the system running this notebook. Most likely, you will have to edit these in order for the model to work correctly.

In [2]:
# data_cache_path
# The directory into which data for the model training should be cached. This
# can be None, but if it is, then the training images will need to be
# regenerated every time the notebook is run.
data_cache_path  = '/data/visual-autolabel/data'

# model_cache_path
# The directory into which to store models that are generated during training.
# This may be None, but if it is, then the best models will not be saved out to
# disk during rounds of training.
model_cache_path = '/data/visual-autolabel/models'

## Initialization

### Libraries

In [3]:
import os, sys, pimms, pandas, json
import numpy as np
import scipy as sp
import nibabel as nib
import pyrsistent as pyr
import neuropythy as ny
import torch, torchvision, torchsummary

import matplotlib as mpl
import matplotlib.pyplot as plt
import ipyvolume as ipv

import visual_autolabel as va

In [4]:
%matplotlib inline

In [5]:
# Additional matplotlib preferences; these are just display preferences.
mpl_font_config = {'family':'sans-serif',
                   'sans-serif':['HelveticaNeue', 'Helvetica', 'Arial'],
                   'size': 10,
                   'weight': 'light'}
mpl.rc('font', **mpl_font_config)
# We want relatively high-res images, especially when saving to disk.
mpl.rcParams['figure.dpi'] = 72*4
mpl.rcParams['savefig.dpi'] = 72*8

### Utilities

In [6]:
def train_until(training_plan, until=None, model_key=None,
                model_cache_path=model_cache_path,
                data_cache_path=data_cache_path,
                create_directories=True,
                create_mode=0o755):
    """Continuously runs the given training plan for models until an interrupt.
        
    Runs training on `'anat'`, `'func'`, and `'both'` models, sequentially,
    using random partitions until a keyboard interrupt is caught, at which
    point a `pandas` dataframe of the results is returned. The partition is
    generated only once per group of model trainings (i.e., per training of an
    anatomical, functional, and combined model).
    
    Parameters
    ----------
    training_plan : list of dicts
        The training-plan to pass to the `visual_autolabel.run_modelplan()`
        function.
    model_key : str or None, optional
        A string that should be appended, as a sub-directory name, to the
        `model_cache_path`; this argument allows one to save model training
        to a specific sub-directory of the `model_cache_path` directory.
    model_cache_path : str, optional
        The cache-path to use for the model training. By default, this is the
        global variable `model_cache_path`, defined above.
    data_cache_path : str, optional
        The cache-path from which data for the model training should be loaded.
        By default, this is the global variable `data_cache_path`, defined
        above.
    until : int or None, optional
        If an integer is provided, then only `until` groups of trainings are
        performed, then the result is returned. If `None`, then the training
        continues until a `KeyboardInterrupt` is caught. The default is `None`.
    create_directories : boolean, optional
        Whether to create cache directories that do not exist (default `True`).
    create_mode : int, optional
        What mode to use when creating directories (default: `0o755`).
    """
    if data_cache_path is Ellipsis:
        data_cache_path = globals()['data_cache_path']
    if model_cache_path is Ellipsis:
        model_cache_path = globals()['model_cache_path']
    if model_key is not None:
        if model_cache_path is None:
            model_cache_path = model_key
        else:
            model_cache_path = os.path.join(model_cache_path, model_key)
    if not os.path.isdir(model_cache_path) and create_directories:
        os.makedirs(model_cache_path, create_mode)
    if not os.path.isdir(data_cache_path) and create_directories:
        os.makedirs(data_cache_path, create_mode)
    training_history = []
    datatype_tr = dict(anat='Anatomical Data Only',
                       func='Functional Data Only',
                       both='Anatomical & Functional Data')
    try:
        print('')
        iterno = 0
        while True:
            if until is not None and iterno >= until: break
            iterno += 1
            # Make one partition for all three minimization types.
            part = va.partition(va.sids, how=(0.8, 0.2))
            pid = va.partition_id(part)
            print('%-15s%70s' % ('Iteration %d' % iterno,
                                 'Partition ID: %s' % pid))
            print('=' * 85)
            for (dtype,dnm) in datatype_tr.items():
                print('')
                print(dnm + ' ' + '-'*(85 - len(dnm) - 1))
                print('')
                t0 = time.time()
                (model, loss, dice) = va.train.run_modelplan(
                    training_plan,
                    partition=part,
                    features=dtype,
                    model_cache_path=model_cache_path,
                    data_cache_path=data_cache_path)
                t1 = time.time()
                row = dict(input=dtype, loss=loss, dice=dice,
                           training_time=(t1-t0))
                training_history.append(row)
                print('')
    except KeyboardInterrupt:
        print('')
        print('KeyboardInterrupt caught; ending training.')
    training_history = ny.to_dataframe(training_history)
    return training_history

In [21]:
def plot_prediction(dataset, k, model,
                    axes=None, figsize=(6,1), dpi=72*4,
                    min_alpha=0.5,
                    channels=(0,1,4,5),
                    round_labels=True):
    """Plots the data, true label, and predicted label (by model) of a dataset.
    
    `plot_prediction(dataset, k, model)` creates a `matplotlib` figure for
    `dataset[k]` (i.e., the `k`th subject/image in `dataset`). The `axes` are
    always returned.
    
    Parameters
    ----------
    dataset : HCPVisualDataset
        The dataset used to plot the predictions. This may alternately be a
        PyTorch dataloader, in which case the dataloader's dataset must be an
        `HCPVisualDataset` object.
    k : int
        The sample number or subject ID to plot. A sample number is just the
        index number for the subject in the dataset; if a number less than 1000
        is given, then it is assumed to eb a subject index, while if it is over
        1000, it is assumed to be a subject ID.
    model : PyTorch Module
        A UNet model or other PyTorch model that makes a segmentation
        of the images from the given `dataset`.
    axes : MatPlotLib axes or `None`, optional
        A set of axes onto which to plot the predictions. Must have a total
        flattened length of 3.
    figsize : tuple, optional
        A tuple of `(width, height)` in inches to use for the figure size. This
        is ignored if `axes` is provided. The default is `(6, 1)`.
    dpi : int, optional
        The number of dots per inch in the output image. If `axes` is provided,
        this option is ignored. The default is `72 * 4`.
    min_alpha : float, optional
        The minimum alpha value to show in the alpha channel of the image.
        Values below this level are replaced by the formula
        `adjusted_value = value * (1 - min_alpha) + min_alpha`. The default is
        `0.5`.
    channels : iterable of ints, optional
        When a dataset whose input images have more than 4 image channels is
        provided (i.e., the `'both'` datasets, which have 4 anatomical and 4
        functional image layers), then this list of 4 channels is used. By
        default this is `(0,1,4,5)`. This option is ignored if the dataset
        contains images with only 4 channels.
    round_labels : boolean, optional
        Whether to round every channel to either 1 or 0 before plotting the
        labels. The default is `True`.
    """
    if k > 1000:
        # We have a subject-ID instead of an index.
        k = np.where(dataset.sids == k)[0]
    (imdat, imlbl) = dataset[k]
    impre = model(imdat[None,:,:,:].float())
    if not model.apply_sigmoid:
        impre = torch.sigmoid(impre)
    impre = dataset.inv_transform(None, impre.detach()[0])
    (imdat, imlbl) = dataset.inv_transform(imdat, imlbl)
    if axes is None:
        (fig,axes) = plt.subplots(1, 3, figsize=figsize, dpi=dpi)
    # with imdat we want to adjust the alpha layer
    imdat = np.array(imdat)
    imdat[:,:,3] = imdat[:,:,3]*(1 - min_alpha) + min_alpha
    for (ax,im,rl) in zip(axes, [imdat,imlbl,impre], [0,0,round_labels]):
        if im.shape[2] > 4: im = im[:,:,:4]
        im = np.clip(im, 0, 1)
        if rl:
            z = im < 0.5
            im[z] = 0
            im[~z] = 1
        ax.imshow(im)
        ax.axis('off')
    return axes

## Training

### Training Plan

First, we define our standard training plan for training models.

In [8]:
training_plan = [
    dict(lr=0.00375, gamma=0.9, num_epochs=10,  bce_weight=0.67),
    dict(lr=0.00250, gamma=0.9, num_epochs=10,  bce_weight=0.33),
    dict(lr=0.00125, gamma=0.9, num_epochs=10,  bce_weight=0.00)]

Additionally, we define a standard set of training and validation subjects. If the files `training_sids.json` and `validate_sids.json` are found in the current directory, they are imported; otherwise, a random partition is made.

In [9]:
if (os.path.isfile('training_sids.json') and
    os.path.isfile('validate_sids.json')):
    with open('training_sids.json', 'r') as fl:
        trn_sids = np.array(json.load(fl))
    with open('validate_sids.json', 'r') as fl:
        val_sids = np.array(json.load(fl))
else:
    (trn_sids, val_sids) = va.partition(va.sids)
    # The following can be used to save these out to files.
    #with open('training_sids.json', 'w') as fl:
    #    json.dump(trn_sids.tolist(), fl)
    #with open('validate_sids.json', 'w') as fl:
    #    json.dump(val_sids.tolist(), fl)

### Simple Training

In [10]:
# Basic options:
opts = dict(partition=(trn_sids, val_sids),
            model_cache_path=model_cache_path,
            data_cache_path=data_cache_path)
# We store the results in these dictionaries.
models = {}
losses = {}
dice = {}
# Run one modelplan for each feature type.
for feat in ('anat', 'func', 'both'):
    print("="*85)
    print(f"Training: {feat}")
    print("")
    r = va.run_modelplan(training_plan,
                         features=feat,
                         **opts)
    print("")
    models[feat] = r[0]
    losses[feat] = r[1]
    dice[feat] = r[2]
print("Training complete.")

Training: anat

epoch    lr     dt[s]   trn bce   trn dice  trn loss   val bce   val dice  val loss
=====--=======--=====-+-========--========--========-+-========--========--========
 1/10  0.00375   50.9 |   12.367     0.888     8.579 |    0.426     0.890     0.579 *
 2/10  0.00337   13.3 |    0.302     0.658     0.419 |    0.301     0.699     0.433 *
 3/10  0.00304   17.2 |    0.245     0.530     0.339 |    0.201     0.444     0.281 *
 4/10  0.00273   21.9 |    0.196     0.409     0.266 |    0.191     0.373     0.251 *
 5/10  0.00246   24.5 |    0.174     0.345     0.230 |    0.182     0.350     0.237 *
 6/10  0.00221   23.2 |    0.156     0.311     0.207 |    0.217     0.345     0.259 
 7/10  0.00199   20.6 |    0.150     0.290     0.196 |    0.144     0.282     0.189 *
 8/10  0.00179   20.8 |    0.142     0.268     0.184 |    0.172     0.328     0.223 
 9/10  0.00161   20.8 |    0.139     0.260     0.178 |    0.164     0.295     0.207 
10/10  0.00145   21.0 |    0.141     0.266   

### Train Continuously

Here, we train the model continuously until an interruption is received (typically a keyboard interruption, which can be sent via the `Kernel > Interrupt Kernel` menu item in Jupyter. This slowly produces a lot of output, but the result, which is returned once the interrupt is sent, will be a `pandas` dataframe of training statistics.

In [ ]:
hist = train_until(training_plan, model_key='2022-04-15_01')

### Visualization of Predictions

In [ ]:
# Parameters
#------------------------------------------------------------------------------
# What feature-set do we want to plot?
features = 'both'
# What model are we plotting the results for?
model = models[features]
# Which subject-IDs (or subject indices, either one) are we plotting?
plot_subs = np.arange(6)
# Training or Validation data?
phase = 'val'
# What datasets are we using? (If None, then makes one using the default
# partition).
datasets = None

# Make the Figure
#------------------------------------------------------------------------------
# Make the datasets if need be.
if datasets is None:
    datasets = va.make_datasets(partition=(trn_sids, val_sids),
                                cache_path=data_cache_path)
# Make the figure and axes that we're going to use.
(fig,axs) = plt.subplots(len(plot_subs), 3, figsize=(6, len(plot_subs)), dpi=72*4)
# And plot each row using the `plot_prediction` function.
for (axrow,idx) in zip(axs, plot_subs):
    plot_prediction(datasets[features][phase], idx, model, axes=axrow)

### Accuracy Comparison

This code needs to be checked and updated.

In [222]:
func_model = trained_models['func'][0]
anat_model = trained_models['anat'][0]

func_ds = datasets['val']['func']
anat_ds = datasets['val']['anat']

def npdice(a,b):
    a = torch.tensor(a.T[:3])
    b = torch.tensor(b.T[:3])
    return 1 - dice_loss(a[None,:,:,:], b[None,:,:,:])
def subdice(ds, k, model):
    sid = ds.sids[k]
    (trnim, ansim) = ds[k]
    preim = torch.sigmoid(model(trnim[None,:,:,:].float()))
    preim = ds.inv_transform(None, preim.detach()[0])
    (trnim, ansim) = ds.inv_transform(trnim, ansim)
    infim = comparison_image(sid)
    priim = companat_image()
    return {'model': npdice(preim, ansim),
            'bayes': npdice(infim, ansim),
            'prior': npdice(priim, ansim)}
def alldice(ds, model):
    dat = [subdice(ds, k, model) for k in range(len(ds.sids))]
    res = []
    for (m,sid) in zip(dat, ds.sids):
        m = {k:v.detach().numpy() for (k,v) in m.items()}
        m['sid'] = sid
        res.append(m)
    return ny.util.to_dataframe(res)

In [223]:
d = alldice(anat_ds, anat_model)

In [224]:
{k: (np.mean(v), np.std(v)/np.sqrt(len(v)), np.median(v), np.percentile(v, 25), np.percentile(v, 75))
 for k in ['model','bayes','prior']
 for v in [d[k].values]}

{'model': (0.7586430125225625,
  0.008477384465692323,
  0.7724644373568392,
  0.7247349369185487,
  0.8021125891625124),
 'bayes': (0.7087995141025225,
  0.009643316660442929,
  0.7249986218068809,
  0.6947594585447514,
  0.7440246912721558),
 'prior': (0.6983875593065093,
  0.0077295654142980575,
  0.7006973293733195,
  0.6724228998098741,
  0.7331302495354244)}

### Hyper-parameter Grid Search

This code needs to be updated.

In [ ]:
grid_params = {
    'dataset': ['both'],
    'batch_size': [2,5,8],
    'lr': [0.01,0.005,0.00375,0.0025,0.00125],
    'gamma': [0.95, 0.9, 0.75, 0.5, 0.25],
    'pretrained_resnet': [True, False]}

grid = []
for (k,vs) in grid_params.items():
    if len(grid) == 0:
        grid = [{k:v} for v in vs]
    else:
        grid = [dict(u, **{k:v}) for u in grid for v in vs]

In [ ]:
import shutil

os.nice(10) # run these nicely!
cpath0 = os.path.join(resnet_path, 'grid')

for (ii,row) in enumerate(grid):
    if ii % 12 == 0:
        print("%3d of %-3d (%6.2f%%)" % (ii+1, len(grid), ii/(len(grid)-1)*100))
    tag = 'dat%04d' % (ii,)
    cpath = os.path.join(cpath0, tag)
    cfile = os.path.join(cpath0, tag + '.json')
    if os.path.isfile(cfile):
        for (k,v) in ny.load(cfile).items():
            row[k] = v
        continue
    mdl0 = resnet_model(pretrained=row['pretrained'])
    (mdl,loss,dice) = plan_to_model(mdl0, [row], cache_path=cpath, logger=None)
    row['tag'] = tag
    row['loss'] = loss
    row['dice'] = dice
    # Clear out the files that we don't need and resave the model and data
    ny.save(cfile, row)
    shutil.move(os.path.join(cpath,  'round01', 'optim000019.pkl'),
                os.path.join(cpath0, 'opt%04d.pkl' % ii))
    shutil.move(os.path.join(cpath,  'round01', 'model000019.pkl'),
                os.path.join(cpath0, 'mdl%04d.pkl' % ii))
    shutil.rmtree(cpath)    
    
grid = ny.to_dataframe(grid)